In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from PIL import Image
from utils import read_mat_file, visualize_sample_dataset, vae_loss
from dataloader import SVHDDataset
import scipy.io

from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from model import VGG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
import os
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
# Train the model
from model import VGG16, VAE
import cv2 

os.environ['TORCH_NNPACK'] = '0'

In [2]:
train_path = '/data/omscs_datasets/train/'
train_mat_file_path = train_path + 'digitStruct.mat'
test_path = '/data/omscs_datasets/train/'
test_mat_file_path = test_path + 'digitStruct.mat'
device = torch.device("cuda")

In [3]:
def detect_digit_from_image_reconstruct(img_path, model, output_image_path, device):
    # Load your model
    #model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    # Load the image (now in color)
    image = cv2.imread(img_path)

    # Define the window size and step size
    win_size = 16
    window_size = (win_size, win_size)
    step_size = 10

    # List to store bounding box coordinates and predictions
    bbox_coords = []

    # Sliding window
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            # Extract and preprocess the window
            window = image[y:y + window_size[1], x:x + window_size[0]]
            processed_window = cv2.resize(window, (32, 32))
            processed_window = transforms.ToTensor()(processed_window)
            processed_window = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(processed_window)
            processed_window = processed_window.unsqueeze(0).to(torch.float32).to(device)
            #print('processed_window:', processed_window.shape)
            # Classify the window
            with torch.no_grad():
                x_reconstructed, mu, log_var  = model(processed_window)
                loss = vae_loss(x_reconstructed, processed_window, mu, log_var)
                print('outputs:', loss)
                print(stop)
                #_, predicted = torch.max(outputs, 1)

            # If it's a digit, save the bounding box coordinates and prediction
            if predicted.item() == 8:
                bbox_coords.append((x, y, x + window_size[0], y + window_size[1], predicted.item()))

    # Draw bounding boxes and text on the original image
    for (x1, y1, x2, y2, prediction) in bbox_coords:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.putText(image, str(prediction), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Save the final image with bounding boxes and text
    cv2.imwrite(output_image_path, image)

In [ ]:
# Hyperparameters
image_channels = 3
h_dim = 1024
z_dim = 32
learning_rate = 1e-3

vae = VAE(image_channels=image_channels, h_dim=h_dim, z_dim=z_dim)
vae = vae.to(device)

In [ ]:
detect_digit_from_image_reconstruct('test1.png', vae, 'output_with_boxes_reconstruct.png', device)